In [1]:
import pandas as pd
import os
import unicodedata
from config import DATA_DIR

In [2]:
csv_path = os.path.join(DATA_DIR, 'clean_scraped_services.csv')
df = pd.read_csv(csv_path)

In [3]:
df.head()

,permite_pedido_anomimo,img_url,service_desc,service_id,ativo,subtheme_name,theme_name,description
0,False,https://sigrc.prefeitura.sp.gov.br/assets/medi...,Adotar cães e gatos,3676,True,Adoção de animais,Animais,O QUE É\nÉ a adoção de cães e gatos alojados n...
1,True,https://sigrc.prefeitura.sp.gov.br/assets/medi...,Pombos - Solicitar vistoria em local infestado,816,True,Animais que transmitem doenças ou risco à saúde,Animais,O QUE É\nÉ a solicitação de vistoria em local ...
2,False,https://sigrc.prefeitura.sp.gov.br/assets/medi...,Morcegos - Reclamar sobre local com morcegos,815,True,Animais que transmitem doenças ou risco à saúde,Animais,"O QUE É\nOrientação, esclarecimento de dúvidas..."
3,True,https://sigrc.prefeitura.sp.gov.br/assets/medi...,Ratos - Solicitar vistoria em local infestado,814,True,Animais que transmitem doenças ou risco à saúde,Animais,O QUE É\nSolicitação de vistoria em local com ...
4,True,https://sigrc.prefeitura.sp.gov.br/assets/medi...,Escorpião - Solicitar vistoria de local com es...,813,True,Animais que transmitem doenças ou risco à saúde,Animais,O QUE É\nÉ a solicitação de vistoria em local ...


In [4]:
exemplo = df.loc[0, 'description']

In [5]:
def remover_acentos(texto: str) -> str:

    nfkd = unicodedata.normalize("NFD", texto)
    texto_sem_acentos = "".join(
        c for c in nfkd if unicodedata.category(c) != "Mn"
    )
    
    return texto_sem_acentos

def clean_chave(chave:str)->str:

    chave = chave.lower().replace(' ', '_').replace('-', '_').replace(',', '_').replace('__', '_').strip()
    chave = remover_acentos(chave)

    return chave

def clean_pedaco(pedaco:str)->str:

    pedaco = pedaco.strip().replace('\xa0', '').replace('clique aqui', '')
    return pedaco


def parse_description(description:str)->dict[str, str]:

    pedacos = description.split('\n')

    atualizacao = pedacos.pop(-1).replace('Atualizado em:', '').strip()
    criacao  = pedacos.pop(-1).replace('Criado em:', '').strip()

    parsed = {
        'dt_atualizacao' : atualizacao,
        'dt_criacao'    : criacao,
    }

    chave = None
    for i, pedaco in enumerate(pedacos):
        if pedaco.isupper():
            chave = clean_chave(pedaco)
        else:
            if chave:
                if chave not in parsed:
                    parsed[chave] = clean_pedaco(pedaco)
                else:
                    parsed[chave] =  parsed[chave] + '***|||***' + clean_pedaco(pedaco)
    
    return parsed


def chunk_header(chunk:str, chunk_key: str, service_name:str, posit:int|None=None)->str:

    header = f"[{chunk_key.upper()} | {service_name}]"

    if posit is not None:
        header = f"[{chunk_key.upper()} | {posit} | {service_name}]"

    return header + '\n' + chunk


def solve_list_chunks(parsed:dict, chave:str, service_name:str, splited:list[str])->None:

    parsed[chave] = []
    posit = 1
    for i, item in enumerate(splited):
        if item.replace('\n', '').strip().endswith(':'):
            next = i+1
            dropar = []
            while next < len(splited) and (splited[next].startswith('-') or splited[next][0].isdigit()):
                dropar.append(next)
                item = item + '\n ' + splited[next]
                next+=1
            for index in sorted(dropar, reverse=True):
                splited.pop(index)
                
        parsed[chave].append(chunk_header(item, chave, service_name, posit))
        posit+=1

def add_header_to_chunks(parsed_chunks:dict[str, str], service_name:str)->dict[str, str]:


    parsed = {}
    #reestruturando os chunks para formato de lista e colocando o header
    for chave, pedacos in parsed_chunks.items():

        splited = pedacos.split('***|||***')
        if len(splited)>1:
            solve_list_chunks(parsed, chave, service_name, splited)
        else:
            parsed[chave] = chunk_header(pedacos, chave, service_name)

    return parsed


def chunk_json_pipeline(description:str, service_name:str)->dict[str, str]:

    parsed = parse_description(description)
    parsed_with_headers = add_header_to_chunks(parsed, service_name)

    return parsed_with_headers


In [6]:
from pprint import pprint

In [7]:
parsed_exemplo = chunk_json_pipeline(exemplo, 'Exemplo de Serviço')
pprint(parsed_exemplo)

{'canais_para_solicitar': ['[CANAIS_PARA_SOLICITAR | 1 | Exemplo de Serviço]\n'
                           'Eletrônico (somente para animais provenientes de '
                           'apreensão de comércio ilegal/animais de raça '
                           'presumida):\n'
                           ' - Portal de Atendimento SP156 (você já está '
                           'aqui).',
                           '[CANAIS_PARA_SOLICITAR | 2 | Exemplo de Serviço]\n'
                           'Presencial:\n'
                           ' -Centro Municipal de Adoção de Cães e Gatos -Rua '
                           'Santa Eulália, 86 - Santana, São Paulo - SP ( para '
                           'ver endereço no mapa).\n'
                           ' -Atendimento de segunda à sexta-feira, das 09h às '
                           '17h; aos sábados e domingos das 09h às 15h, exceto '
                           'feriados.'],
 'dt_atualizacao': '[DT_ATUALIZACAO | Exemplo de Serviço]\n20/10/2025'

In [8]:
type(parsed_exemplo['canais_para_solicitar'][0])

str

In [9]:
print(parsed_exemplo['canais_para_solicitar'][0])

[CANAIS_PARA_SOLICITAR | 1 | Exemplo de Serviço]
Eletrônico (somente para animais provenientes de apreensão de comércio ilegal/animais de raça presumida):
 - Portal de Atendimento SP156 (você já está aqui).


In [10]:
print(parsed_exemplo['canais_para_solicitar'][1])

[CANAIS_PARA_SOLICITAR | 2 | Exemplo de Serviço]
Presencial:
 -Centro Municipal de Adoção de Cães e Gatos -Rua Santa Eulália, 86 - Santana, São Paulo - SP ( para ver endereço no mapa).
 -Atendimento de segunda à sexta-feira, das 09h às 17h; aos sábados e domingos das 09h às 15h, exceto feriados.


In [11]:
print(parsed_exemplo['requisitos_documentos_e_informacoes'][2])

[REQUISITOS_DOCUMENTOS_E_INFORMACOES | 3 | Exemplo de Serviço]
Importante:Levar coleira (no caso de adoção de cães) ou caixa de transporte (no caso de gatos) para o transporte seguro do animal.


In [12]:
def flaten_chunks(parsed_json:dict)->list[str]:

    flatened = []
    for pedacos in parsed_json.values():
        if isinstance(pedacos, list):
            for pedaco in pedacos:
                flatened.append(pedaco)
        else:
            flatened.append(pedacos)
    
    return flatened

In [13]:
chunks = flaten_chunks(parsed_exemplo)

In [14]:
chunks[0]

'[DT_ATUALIZACAO | Exemplo de Serviço]\n20/10/2025'

In [15]:
df.head()

,permite_pedido_anomimo,img_url,service_desc,service_id,ativo,subtheme_name,theme_name,description
0,False,https://sigrc.prefeitura.sp.gov.br/assets/medi...,Adotar cães e gatos,3676,True,Adoção de animais,Animais,O QUE É\nÉ a adoção de cães e gatos alojados n...
1,True,https://sigrc.prefeitura.sp.gov.br/assets/medi...,Pombos - Solicitar vistoria em local infestado,816,True,Animais que transmitem doenças ou risco à saúde,Animais,O QUE É\nÉ a solicitação de vistoria em local ...
2,False,https://sigrc.prefeitura.sp.gov.br/assets/medi...,Morcegos - Reclamar sobre local com morcegos,815,True,Animais que transmitem doenças ou risco à saúde,Animais,"O QUE É\nOrientação, esclarecimento de dúvidas..."
3,True,https://sigrc.prefeitura.sp.gov.br/assets/medi...,Ratos - Solicitar vistoria em local infestado,814,True,Animais que transmitem doenças ou risco à saúde,Animais,O QUE É\nSolicitação de vistoria em local com ...
4,True,https://sigrc.prefeitura.sp.gov.br/assets/medi...,Escorpião - Solicitar vistoria de local com es...,813,True,Animais que transmitem doenças ou risco à saúde,Animais,O QUE É\nÉ a solicitação de vistoria em local ...


In [16]:
all_chunks = []
metadata_list = []
for i, row in df.iterrows():
    description = row['description']
    service_name = row['service_desc']
    parsed_chunks = chunk_json_pipeline(description, service_name)
    chunks = flaten_chunks(parsed_chunks)
    all_chunks.extend(chunks)
    for chunk in chunks:
        metadata = {
            "service_name": service_name,
            "service_id" : row['service_id'],
            "theme" : row['theme_name'],
            "subtheme" : row['subtheme_name'],
            "content" : chunk
        }
        metadata_list.append(metadata)

In [17]:
assert len(all_chunks) == len(metadata_list)

In [18]:
import json

In [19]:
dados = {
    "chunks": all_chunks,
    "metadata": metadata_list}

In [20]:
json_path = os.path.join(DATA_DIR, 'chunks_metadata.json')
with open(json_path, 'w') as f:
    json.dump(dados, f, ensure_ascii=False, indent=4)